# Scraping for Categories, Winners, and Nominees for the 2022 GRAMMYs
- I'm one of those people who always claims to be really into music yet the only thing that i know about
the 2022 GRAMMYs is that [Tyler's](https://www.google.com/search?gs_ssp=eJzj4tLP1TdILkopySo3YPQSLKnMSS1SKMlIVUguSk0syS8CAKRdCss&q=tyler+the+creator&oq=tyler&aqs=chrome.1.69i57j46i39j46i67i433j46i67j46i433i512j69i61l3.3555j0j7&sourceid=chrome&ie=UTF-8) album won 😂. And if you are seeing this you probably don't know much about the event too. Let's fix that so that before our friends start a conversation we can't keep up with. We'll would already know so much we would be the ones starting the conversation like
![How you doing](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQZTc4hymVFN3dgAk1NZfGKBtW3oPOXz3rgbQ&usqp=CAU)

### Pick a website and describe your objective

- Browse through different sites and pick on to scrape. Check the "Project Ideas" section for inspiration.
- Identify the information you'd like to scrape from the site. Decide the format of the output CSV file.
- Summarize your project idea and outline your strategy in a Juptyer notebook. Use the "New" button above.


In [95]:
# Website chosen (URL)
# https://www.grammy.com/news/2022-grammys-complete-winners-nominees-nominations-list

#load libraries
import requests
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TARI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [96]:
# Download the Grammy page
url = "https://www.grammy.com/news/2022-grammys-complete-winners-nominees-nominations-list"
response = requests.get(url)

# Check for error in downloading page
response.raise_for_status()

In [97]:
#size of the page
len(response.text)

368073

In [98]:
page_contents = response.text

### Use the requests library to download web pages

- Inspect the website's HTML source and identify the right URLs to download.
- Download and save web pages locally using the requests library.
- Create a function to automate downloading for different topics/search queries.

In [99]:
# parse the pages_contents as html file
doc = BeautifulSoup(page_contents, 'html.parser')


## inspected website and found out that all the data that i will be working with in the div.prose class

In [100]:
grammydets = doc.find_all('div', {'class': 'prose'})

In [101]:
# Categories
categories_tags = grammydets[1].find_all('h1')
len(categories_tags)

26

In [102]:
categories = []
for i in range(len(categories_tags)):
  categories.append(categories_tags[i].text)
categories
len(categories)

26

# IT Seems like all the awards are in bold text format

In [103]:
# awards 
strong_tags = doc.select('p > strong')
awards = []
for i in range(0, len(strong_tags), 1):
  match = re.match('\d\.', strong_tags[i].text)
  match2 = re.match('\d\d\.', strong_tags[i].text)
  if match or match2 or match:
    awards.append(strong_tags[i].text[3:].strip())
  else:
    continue

In [104]:
awards

['Record Of The Year',
 'Album Of The Year',
 'Song Of The Year',
 'Best New Artist',
 'Best Pop Solo Performance',
 'Best Pop Duo/Group Performance',
 'Best Traditional Pop Vocal Album',
 'Best Pop Vocal Album',
 'Best Dance/Electronic Recording',
 'Best Dance/Electronic Music Album',
 'Best Contemporary Instrumental Album',
 'Best Rock Performance',
 'Best Metal Performance',
 'Best Rock Song',
 'Best Rock Album',
 'Best Alternative Music Album',
 'Best R&B Performance',
 'Best Traditional R&B Performance',
 'Best R&B Song',
 'Best Progressive R&B Album',
 'Best R&B Album',
 'Best Rap Performance',
 'Best Melodic Rap Performance',
 'Best Rap Song',
 'Best Rap Album',
 'Best Country Solo Performance',
 'Best Country Duo/Group Performance',
 'Best Country Song',
 'Best Country Album',
 'Best New Age Album',
 'Best Improvised Jazz Solo',
 'Best Jazz Vocal Album',
 'Best Jazz Instrumental Album',
 'Best Large Jazz Ensemble Album',
 'Best Latin Jazz Album',
 'Best Gospel Performance/Song'

In [105]:
len(awards)

85

# OH UH the's supposed to be 86. I know that much
- let's check the url to see what's wrong
- so award 44 is not in bold that is not enclosed in a strong tag. Nah smalls we go run am

In [106]:
p_tags = grammydets[1].find_all('p')
fourtyfour = ['']
for i in range(0, len(p_tags)):
  match3 = re.match(r'^\*\*\d\d', p_tags[i].text)
  if match3:
    fourtyfour.append(p_tags[i].text)
fourtyfour = fourtyfour[1][5: 58].strip()

In [107]:
fourtyfour

'Best Regional Mexican Music Album (Including Tejano)'

In [108]:
awards.insert(43, fourtyfour)

In [109]:

len(awards)

86

In [110]:
winners = []
tie_list1 = []
tie_list2 = []

for i in range(0, len(strong_tags), 1):
    match = re.search('WINNER', strong_tags[i].text)
    match2 = re.search('Sour Olivia Rodrigo', strong_tags[i].text)
    match3 = re.search('WINNNER', strong_tags[i].text)
    match4 = re.search('TIE', strong_tags[i].text)
    match5 = re.search('Tie', strong_tags[i].text)
    if match:
      winners.append(strong_tags[i].text)
    elif match2:
      if strong_tags[i] not in winners:
        winners.append(strong_tags[i].text)
    elif match3:
      winners.append(strong_tags[i].text)
    elif match4:
      tie_list1.extend(sent_tokenize(strong_tags[i].text))
      if tie_list1 not in winners:
        winners.append(tie_list1)
    elif match5:
      tie_list2.extend(sent_tokenize(strong_tags[i].text))
      if tie_list2 not in winners:
        winners.append(tie_list2)
    else:
      continue

In [111]:
len(winners)

86

Data gotten manually from url 2

In [112]:
for i in range(len(categories)):
  print(f"{i}: ", categories[i])

0:  General Field
1:  Pop
2:  Dance/Electronic Music
3:  Contemporary Instrumental Music
4:  Rock
5:  Alternative
6:  R&B
7:  Rap
8:  Country
9:  New Age
10:  Jazz
11:  Gospel/Contemporary Christian Music
12:  Latin
13:  American Roots Music
14:  Reggae
15:  Global Music
16:  Children's
17:  Spoken Word
18:  Comedy
19:  Musical Theater
20:  Music for Visual Media
21:  Composing/Arranging
22:  Package, Notes, and Historical
23:  Production
24:  Classical
25:  Music Video/Film


In [113]:
no_of_awards = [4, 4, 2, 1, 4, 1, 5, 4, 4, 1, 5, 5, 5, 8, 1, 2, 1, 1, 1, 1, 3, 3, 4, 6, 8, 2]
for i in range(len(no_of_awards)):
  print(f"{i}: ", no_of_awards[i])

0:  4
1:  4
2:  2
3:  1
4:  4
5:  1
6:  5
7:  4
8:  4
9:  1
10:  5
11:  5
12:  5
13:  8
14:  1
15:  2
16:  1
17:  1
18:  1
19:  1
20:  3
21:  3
22:  4
23:  6
24:  8
25:  2


In [114]:
categories_dict = {
    'categories':categories,
    'no_of_awards':no_of_awards,
}

In [115]:
df = pd.DataFrame(categories_dict)

In [116]:
len(awards)


86

In [117]:
awards_dict = {
    'awards': awards,
    'winners': winners
}


In [118]:
df_awards =  pd.DataFrame(awards_dict)
df_awards

,awards,winners
0,Record Of The Year,"Leave The Door Open - WINNERSilk SonicDernst ""..."
1,Album Of The Year,"We Are - WINNERJon BatisteCraig Adams, David G..."
2,Song Of The Year,"Leave The Door Open - WINNERBrandon Anderson, ..."
3,Best New Artist,Olivia Rodrigo - WINNER
4,Best Pop Solo Performance,drivers license - WINNEROlivia Rodrigo
...,...,...
81,Best Classical Solo Vocal Album,Mythologies - WINNERSangeeta Kaur & Hila Plitm...
82,Best Classical Compendium,Women Warriors - The Voices Of Change - WINNER...
83,Best Contemporary Classical Composition,"Shaw: Narrow Sea - WINNERCaroline Shaw, compos..."
84,Best Music Video,"Freedom - WINNERJon BatisteAlan Ferguson, vide..."


In [119]:
df.to_csv("categories.csv", index=None, mode='w')
df_awards.to_csv("awards.csv", index=None, mode='w')
